###### Install Dependencies

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 30.7 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow keras

###### Set Directory Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Full path of the data to train the model
_DATA = '/content/drive/MyDrive/Data/delaney-processed.csv'

#### Explore Data

In [ ]:
import pandas as pd

df = pd.read_csv(_DATA)

In [ ]:
df.head()

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1


# Model(s)

#### 1st Model: ANN, RNN, LSTM and Reinforcement

In [ ]:
import keras
import numpy as np
import pandas as pd
import os
from keras.layers import Input, Dense, Concatenate, Lambda, Reshape, GRU, LSTM
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import mean_squared_error
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
import tensorflow.compat.v1 as tf
from tensorflow import keras
from keras.layers import Flatten
import random

# Path and filename to save the model
MODEL_DIRECTORY = 'saved_model'
MODEL_FILENAME = 'ann_model_1.h5'
MODEL_PATH = os.path.join(MODEL_DIRECTORY, MODEL_FILENAME)

df = pd.read_csv(_DATA)
df = df.rename(columns={'Compound ID': 'id', 'ESOL predicted log solubility in mols per litre': 'binding_affinity'})
df = df[['id', 'smiles', 'binding_affinity']]
df = df.dropna()

X = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]

X = np.array([list(AllChem.GetMorganFingerprintAsBitVect(mol, 2)) for mol in X])

X_train, X_test, y_train, y_test = train_test_split(X, df['binding_affinity'], test_size=0.2)

# input_shape is the length of the fingerprint vector
input_shape = (X.shape[1],)

# 1: ANN:
# Create input layer with specified shape
inputs = Input(shape=input_shape, dtype=tf.int32)

# Add backpropagation layer
# Convert the int32 tensor to float32
backpropagation = Lambda(lambda x: tf.cast(x, tf.float32))(inputs)

x = Dense(512, activation='relu')(inputs)
x = Concatenate()([x, backpropagation]) # Concatenate the two tensors
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)


# Add flatten layer
flatten = Flatten()(x)

# Add reshape layer(s)
reshape = Reshape((1, 64))(flatten)


# 2: RNN:
# Add RNN layer(s)
rnn1 = GRU(128, input_shape=(None, 64), batch_size=1)(reshape)
rnn2 = GRU(128, input_shape=(None, 64), batch_size=1)(reshape)
rnn3 = GRU(128, input_shape=(None, 64), batch_size=1)(reshape)
rnn_layers = [rnn1, rnn2, rnn3]

# 3: LSTM:
# Add LSTM layer(s)
lstm1 = LSTM(128, input_shape=(None, 64), batch_size=1)(reshape)
lstm2 = LSTM(128, input_shape=(None, 64), batch_size=1)(reshape)
lstm3 = LSTM(128, input_shape=(None, 64), batch_size=1)(reshape)
lstm_layers = [lstm1, lstm2, lstm3]

# 4: Reinforcement:
class EpsilonGreedyPolicy(object):
    def __init__(self, epsilon):
        self.epsilon = epsilon

    def __call__(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, 1)
        else:
            return np.argmax(state)

class ActorCritic(object):
    def __init__(self, policy, critic):
        self.policy = policy
        self.critic = critic

    def __call__(self, state):
        return self.policy(state), self.critic(state)

state = np.array([1, 2, 3])

policy = EpsilonGreedyPolicy(epsilon=0.1)
critic = Lambda(lambda x: x)
agent = ActorCritic(policy, critic)

action, value = agent(state)

reinforcement_layers = [policy, critic]


# Combine all layers (RNN, LSTM, and reinforcement layers)
combined_layers = rnn_layers + lstm_layers
combined_model = Concatenate()(combined_layers)

# Add output layer with single unit and linear
outputs = Dense(1, activation='linear')(combined_model)

# Create model with input and output layer
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=1e-3), loss=mean_squared_error)

model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1)

print(f'Saving trained model to {MODEL_PATH}...')
model.save(MODEL_PATH)

print(f'Loading trained model from {MODEL_PATH}...')
model = load_model(MODEL_PATH)

smiles = ['COC1=C(C=C(C=C1)C(=O)O)O']

X_new = np.array(tf.stack([list(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), 2)) for smi in smiles]))

y_pred = model.predict(X_new)

print('Predicted binding affinity:', y_pred[0][0])

Epoch 1/100
29/29 [==============================] - 15s 14ms/step - loss: 3.8432
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 0.8393
Epoch 3/100
29/29 [==============================] - 0s 12ms/step - loss: 0.5540
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3307
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2051
Epoch 6/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1882
Epoch 7/100
29/29 [==============================] - 0s 13ms/step - loss: 0.1612
Epoch 8/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1683
Epoch 9/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1108
Epoch 10/100
29/29 [==============================] - 0s 13ms/step - loss: 0.0849
Epoch 11/100
29/29 [==============================] - 0s 12ms/step - loss: 0.0943
Epoch 12/100
29/29 [==============================] - 0s 12ms/step - loss: 0.0900
Epoch 13/100
29/29 [====

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 2s 2s/step
Predicted binding affinity: -2.5836887


#### 2nd Model: Conditional Variational Autoencoder

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split

# Path and filename to save the model
MODEL_DIRECTORY = 'saved_model'
MODEL_FILENAME = 'cvae_model_1.h5'
MODEL_PATH = os.path.join(MODEL_DIRECTORY, MODEL_FILENAME)

# Define the desired output dimension for the CVAE model
MODEL_CVAE_OUTPUT_DIM = 1

df = pd.read_csv(_DATA)
df = df.rename(columns={'Compound ID': 'id', 'ESOL predicted log solubility in mols per litre': 'binding_affinity'})
df = df[['id', 'smiles', 'binding_affinity']]
df = df.dropna()

# Scale the target variable
y_train_scaled = -(df['binding_affinity'] - df['binding_affinity'].min()) / (df['binding_affinity'].max() - df['binding_affinity'].min())

X = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]
X = np.array([list(AllChem.GetMorganFingerprintAsBitVect(mol, 2)) for mol in X])

X_train, X_test, y_train, y_test = train_test_split(X, y_train_scaled, test_size=0.2)

# Define input_shape and encoding_dim
input_shape = (X.shape[1],)
encoding_dim = 128
num_classes = 1  # Number of classes, in this case, 1 for binding affinity

# Create input layers for features and class labels
input_features = Input(shape=input_shape)
input_labels = Input(shape=(num_classes,))

# Concatenate features and labels
concatenated_input = Concatenate(axis=-1)([input_features, input_labels])

# Add encoding layers
h1 = Dense(512, activation='relu')(concatenated_input)
h2 = Dense(256, activation='relu')(h1)
h3 = Dense(128, activation='relu')(h2)
z_mean = Dense(encoding_dim)(h3)
z_log_var = Dense(encoding_dim)(h3)

# Add sampling layer (e.g., using the reparameterization trick)
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.keras.backend.shape(z_mean)[0]
    dim = encoding_dim
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim), mean=0.0, stddev=1.0)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

# Name the Lambda layer explicitly
z = Lambda(sampling, output_shape=(encoding_dim,), name='cvae_lambda_layer')([z_mean, z_log_var])

# Define the decoder layers and output
decoding_1 = Dense(128, activation='relu')
decoding_2 = Dense(256, activation='relu')
decoding_3 = Dense(512, activation='relu')
decoding_4 = Dense(input_shape[0], activation='linear')  # Changed activation to 'linear' for continuous output

# Connect the encoding and decoding layers to build the model
decoder_input = Input(shape=(encoding_dim,))
decoder_input_concatenated = Concatenate(axis=-1)([decoder_input, input_labels])
decoder_layer1 = decoding_1(decoder_input_concatenated)
decoder_layer2 = decoding_2(decoder_layer1)
decoder_layer3 = decoding_3(decoder_layer2)
decoder_output = decoding_4(decoder_layer3)

decoder = Model([decoder_input, input_labels], decoder_output)

outputs = decoder([z, input_labels])

# Reverse the scaling during loss calculation
reconstruction_loss = tf.keras.losses.mse(input_features, outputs) * (y_train.max() - y_train.min()) + y_train.min()
kl_loss = -0.5 * tf.keras.backend.mean(1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var), axis=-1)
cvae_loss = tf.keras.backend.mean(reconstruction_loss + kl_loss)

# Compile the full CVAE model
cvae = Model([input_features, input_labels], outputs)
cvae.add_loss(cvae_loss)
cvae.compile(optimizer=Adam(learning_rate=0.001))

# Adjust the output dimension of the CVAE model
# Modify the output layer to produce a single output
# For example, change from Dense(2048, activation='linear') to Dense(1, activation='linear')
input_layer_cvae = Input(shape=input_shape)  # Assuming you have input_shape_cvae defined elsewhere
hidden_layer_cvae = Dense(512, activation='relu')(input_layer_cvae)
# Adjusted output layer
output_layer_cvae = Dense(MODEL_CVAE_OUTPUT_DIM, activation='linear')(hidden_layer_cvae)
MODEL_CVAE = Model(inputs=input_layer_cvae, outputs=output_layer_cvae)
MODEL_CVAE.compile(optimizer='adam', loss='mean_squared_error')

# Train the CVAE model
history = cvae.fit([X_train, y_train], batch_size=128, epochs=50, shuffle=True, validation_data=([X_test, y_test], None))

# Save the trained model
if not os.path.exists(MODEL_DIRECTORY):
    os.makedirs(MODEL_DIRECTORY)

# Save the trained model in the native Keras format
cvae.save(MODEL_PATH)

# Load the saved model
loaded_model = tf.keras.models.load_model(MODEL_PATH)

# Make a prediction for a single molecule
#smiles = 'CC1=NC(=C2C(=N1)C(=O)N(C(=S)N2C)C)N'
smiles = 'COC1=C(C=C(C=C1)C(=O)O)O'
mol = Chem.MolFromSmiles(smiles)
fp = np.array(list(AllChem.GetMorganFingerprintAsBitVect(mol, 2)))
fp = np.reshape(fp, (1, fp.shape[0]))

# Prepare the labels (y) for prediction
label = np.array([[0]])  # Adjust this based on your label format

# Reverse the scaling for prediction
prediction = loaded_model.predict([fp, label]) * (y_train.max() - y_train.min()) + y_train.min()

# Print the predicted binding affinity
print('Predicted binding affinity: ', prediction[0][0])

Epoch 1/50
8/8 [==============================] - 5s 224ms/step - loss: -0.9819 - val_loss: -0.9872
Epoch 2/50
8/8 [==============================] - 1s 69ms/step - loss: -0.9873 - val_loss: -0.9880
Epoch 3/50
8/8 [==============================] - 1s 66ms/step - loss: -0.9878 - val_loss: -0.9882
Epoch 4/50
8/8 [==============================] - 1s 70ms/step - loss: -0.9883 - val_loss: -0.9883
Epoch 5/50
8/8 [==============================] - 1s 68ms/step - loss: -0.9880 - val_loss: -0.9885
Epoch 6/50
8/8 [==============================] - 1s 66ms/step - loss: -0.9884 - val_loss: -0.9887
Epoch 7/50
8/8 [==============================] - 1s 73ms/step - loss: -0.9887 - val_loss: -0.9889
Epoch 8/50
8/8 [==============================] - 1s 66ms/step - loss: -0.9888 - val_loss: -0.9890
Epoch 9/50
8/8 [==============================] - 1s 70ms/step - loss: -0.9892 - val_loss: -0.9892
Epoch 10/50
8/8 [==============================] - 1s 65ms/step - loss: -0.9891 - val_loss: -0.9892
Epoch 11

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 235ms/step
Predicted binding affinity:  -0.99767375


#### 3rd Model: Stacking Model

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from keras.models import load_model
from sklearn.ensemble import RandomForestRegressor

# Load data
df = pd.read_csv(_DATA)
df = df.rename(columns={'Compound ID': 'id', 'ESOL predicted log solubility in mols per litre': 'binding_affinity'})
df = df[['id', 'smiles', 'binding_affinity']]
df = df.dropna()

# Extract features X and target variable y
X = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]
X = np.array([list(AllChem.GetMorganFingerprintAsBitVect(mol, 2)) for mol in X])
y = df['binding_affinity']

# Load models
model_1 = load_model('saved_model/ann_model_1.h5')
model_2 = load_model('saved_model/cvae_model_1.h5')

# Generate placeholder labels for the entire dataset
num_samples = len(X)
placeholder_labels = np.zeros((num_samples, 1))  # Assuming one class label

# Predictions for the entire dataset using both models
y_pred_model_1 = model_1.predict(X)
y_pred_model_2 = model_2.predict([X, placeholder_labels])

# Combine predictions from both models as features
X_stacked = np.hstack((y_pred_model_1, y_pred_model_2))

# Train a simple stacking model (you can use any other algorithm as well)
stacking_model = RandomForestRegressor(n_estimators=1000, random_state=42)
stacking_model.fit(X_stacked, y)

# Load new molecules and preprocess them
new_smiles = ['COC1=C(C=C(C=C1)C(=O)O)O']  # Example new molecule
X_new = np.array([list(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), 2)) for smi in new_smiles])

# Make predictions for the new molecules using both models
y_pred_model_1_new = model_1.predict(X_new)
y_pred_model_2_new = model_2.predict([X_new, np.zeros((len(X_new), 1))])  # Placeholder labels for new molecules

# Combine predictions from both models as features for new molecules
X_new_stacked = np.hstack((y_pred_model_1_new, y_pred_model_2_new))

# Make predictions using the stacking model
y_pred_new = stacking_model.predict(X_new_stacked)

# Print the predicted binding affinity for the new molecules
print("Predicted binding affinity for new molecules:", y_pred_new)

1/1 [==============================] - 0s 42ms/step
Predicted binding affinity for new molecules: [-3.547017]


In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from keras.models import load_model
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

# Load data
df = pd.read_csv(_DATA)
df = df.rename(columns={'Compound ID': 'id', 'ESOL predicted log solubility in mols per litre': 'binding_affinity'})
df = df[['id', 'smiles', 'binding_affinity']]
df = df.dropna()

# Extract features X and target variable y
X = [Chem.MolFromSmiles(smiles) for smiles in df['smiles']]
X = np.array([list(AllChem.GetMorganFingerprintAsBitVect(mol, 2)) for mol in X])
y = df['binding_affinity']

# Load models
model_1 = load_model('saved_model/ann_model_1.h5')
model_2 = load_model('saved_model/cvae_model_1.h5')

# Generate placeholder labels for the entire dataset
num_samples = len(X)
placeholder_labels = np.zeros((num_samples, 1))  # Assuming one class label

# Predictions for the entire dataset using both models
y_pred_model_1 = model_1.predict(X)
y_pred_model_2 = model_2.predict([X, placeholder_labels])

# Combine predictions from both models as features
X_stacked = np.hstack((y_pred_model_1, y_pred_model_2))

# Ensemble Method 1: Gradient Boosting
gradient_boosting_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gradient_boosting_model.fit(X_stacked, y)

# Ensemble Method 2: XGBoost
xgboost_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgboost_model.fit(X_stacked, y)

# Load new molecules and preprocess them
new_smiles = ['COC1=C(C=C(C=C1)C(=O)O)O']  # Example new molecule
X_new = np.array([list(AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi), 2)) for smi in new_smiles])

# Make predictions for the new molecules using both models
y_pred_model_1_new = model_1.predict(X_new)
y_pred_model_2_new = model_2.predict([X_new, np.zeros((len(X_new), 1))])  # Placeholder labels for new molecules

# Combine predictions from both models as features for new molecules
X_new_stacked = np.hstack((y_pred_model_1_new, y_pred_model_2_new))

# Make predictions using the ensemble models
y_pred_new_gb = gradient_boosting_model.predict(X_new_stacked)
y_pred_new_xgb = xgboost_model.predict(X_new_stacked)

# Print the predicted binding affinity for the new molecules using Gradient Boosting
print("Predicted binding affinity for new molecules (Gradient Boosting):", y_pred_new_gb)

# Print the predicted binding affinity for the new molecules using XGBoost
print("Predicted binding affinity for new molecules (XGBoost):", y_pred_new_xgb)

1/1 [==============================] - 0s 36ms/step
Predicted binding affinity for new molecules (Gradient Boosting): [-3.61350326]
Predicted binding affinity for new molecules (XGBoost): [-3.3443282]
